In [1]:
# Import relevant modules
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
from datetime import date, timedelta

In [2]:
#Read file
map_file = Dataset('SOCATv2022_ESACCI.nc', 'r')

In [4]:
# Print data set variables
print(map_file.variables.keys())

dict_keys(['xlon', 'ylat', 'tmnth', 'tmnth_bnds', 'count_ncruise', 'fco2_count_nobs', 'fco2_ave_weighted', 'fco2_ave_unwtd', 'fco2_min_unwtd', 'fco2_max_unwtd', 'fco2_std_weighted', 'fco2_std_unwtd', 'sst_count_nobs', 'sst_ave_weighted', 'sst_ave_unwtd', 'sst_min_unwtd', 'sst_max_unwtd', 'sst_std_weighted', 'sst_std_unwtd', 'salinity_count_nobs', 'salinity_ave_weighted', 'salinity_ave_unwtd', 'salinity_min_unwtd', 'salinity_max_unwtd', 'salinity_std_weighted', 'salinity_std_unwtd', 'lat_offset_unwtd', 'lon_offset_unwtd', 'count_ncruise_reanalysed', 'count_nobs_reanalysed', 'fco2_reanalysed_max', 'pco2_reanalysed_max', 'fco2_reanalysed_min', 'pco2_reanalysed_min', 'fco2_reanalysed_std_weighted', 'pco2_reanalysed_std_weighted', 'fco2_reanalysed_ave_unwtd', 'pco2_reanalysed_ave_unwtd', 'fco2_reanalysed_std_unwtd', 'pco2_reanalysed_std_unwtd', 'fco2_reanalysed_ave_weighted', 'pco2_reanalysed_ave_weighted', 'weighted_dfco2', 'weighted_dpco2', 'weighted_dT', 'sst_reynolds'])

In [3]:
# Print time (months) metadata
print(map_file.variables['tmnth'])

<class 'netCDF4._netCDF4.Variable'>
float64 tmnth(tmnth)
    units: days since 1970-01-01
    axis: T
    bounds: tmnth_bnds
    time_origin: 01-JAN-1970
    standard_name: time
unlimited dimensions: tmnth
current shape = (624,)
filling on, default _FillValue of 9.969209968386869e+36 used


In [10]:
start_date = date(1970,1,1) # Set reference date
num_days = np.ma.getdata(map_file.variables['tmnth'])[240] # Finding index of start date (16/01/1990)
new_date = start_date + timedelta(days=num_days) # Produce date
print(new_date) # Print date as check

num_days = np.ma.getdata(map_file.variables['tmnth'])[-1] # Finding index of final date (16/12/2021) [-1 = last value]
new_date = start_date + timedelta(days=num_days) # Produce date
print(new_date) # Print date as check

1990-01-16
2021-12-16


In [5]:
# Print latitude values
print(map_file.variables['ylat'][:])
# Print longitude values
print(map_file.variables['xlon'][:])

masked_array(data=[-89.5, -88.5, -87.5, -86.5, -85.5, -84.5, -83.5, -82.5,
                   -81.5, -80.5, -79.5, -78.5, -77.5, -76.5, -75.5, -74.5,
                   -73.5, -72.5, -71.5, -70.5, -69.5, -68.5, -67.5, -66.5,
                   -65.5, -64.5, -63.5, -62.5, -61.5, -60.5, -59.5, -58.5,
                   -57.5, -56.5, -55.5, -54.5, -53.5, -52.5, -51.5, -50.5,
                   -49.5, -48.5, -47.5, -46.5, -45.5, -44.5, -43.5, -42.5,
                   -41.5, -40.5, -39.5, -38.5, -37.5, -36.5, -35.5, -34.5,
                   -33.5, -32.5, -31.5, -30.5, -29.5, -28.5, -27.5, -26.5,
                   -25.5, -24.5, -23.5, -22.5, -21.5, -20.5, -19.5, -18.5,
                   -17.5, -16.5, -15.5, -14.5, -13.5, -12.5, -11.5, -10.5,
                    -9.5,  -8.5,  -7.5,  -6.5,  -5.5,  -4.5,  -3.5,  -2.5,
                    -1.5,  -0.5,   0.5,   1.5,   2.5,   3.5,   4.5,   5.5,
                     6.5,   7.5,   8.5,   9.5,  10.5,  11.5,  12.5,  13.5,
                    14.5,

In [12]:
# Print metadata for pco2 values
print(map_file.variables['pco2_reanalysed_ave_weighted'])

<class 'netCDF4._netCDF4.Variable'>
float32 pco2_reanalysed_ave_weighted(tmnth, ylat, xlon)
    _FillValue: -999.0
    units: uatm
    missing_value: -999.0
    valid_min: 0.0
    valid_max: 1000000.0
    scale_factor: 1.0
    add_offset: 0.0
    standard_name: weighted_pCO2_Tym
    long_name: CO2 partial pressure using OC-FLUX methodology cruise-weighted.
unlimited dimensions: tmnth
current shape = (624, 180, 360)
filling on

In [13]:
# Get pco2 data for identified time indexes 
pco2_reanalysed = np.ma.getdata(map_file.variables['pco2_reanalysed_ave_weighted'][240,:,:])

# Replace mask values with NaN
pco2_reanalysed[pco2_reanalysed <= -998] = np.nan

# Print shape of data as check
print(pco2_reanalysed.shape)

In [16]:
# Loop over identified time indexes
for i in range(240, map_file.variables['tmnth'].shape[0]):
    
    # Get month pco2 data
    one_month = np.ma.getdata(map_file.variables['pco2_reanalysed_ave_weighted'][i,:,:])
    # Replace mask values with NaN
    one_month[one_month <= -998] = np.nan
    # Flip data as it is reversed in original data
    one_month = np.flip(one_month, axis=1)
    # Rotate data twice as data is upside down
    one_month = np.rot90(one_month,2)
    
    # If index is start value initialise 3D array to stored monthly data
    if i == 240:
        pco2_stack = one_month
    # Else append monthly data to 3D array
    else:
        pco2_stack = np.dstack((pco2_stack,one_month))
    
    # Print data shape as check
    print(i, pco2_stack.shape)

240 (180, 360)
241 (180, 360, 2)
242 (180, 360, 3)
243 (180, 360, 4)
244 (180, 360, 5)
245 (180, 360, 6)
246 (180, 360, 7)
247 (180, 360, 8)
248 (180, 360, 9)
249 (180, 360, 10)
250 (180, 360, 11)
251 (180, 360, 12)
252 (180, 360, 13)
253 (180, 360, 14)
254 (180, 360, 15)
255 (180, 360, 16)
256 (180, 360, 17)
257 (180, 360, 18)
258 (180, 360, 19)
259 (180, 360, 20)
260 (180, 360, 21)
261 (180, 360, 22)
262 (180, 360, 23)
263 (180, 360, 24)
264 (180, 360, 25)
265 (180, 360, 26)
266 (180, 360, 27)
267 (180, 360, 28)
268 (180, 360, 29)
269 (180, 360, 30)
270 (180, 360, 31)
271 (180, 360, 32)
272 (180, 360, 33)
273 (180, 360, 34)
274 (180, 360, 35)
275 (180, 360, 36)
276 (180, 360, 37)
277 (180, 360, 38)
278 (180, 360, 39)
279 (180, 360, 40)
280 (180, 360, 41)
281 (180, 360, 42)
282 (180, 360, 43)
283 (180, 360, 44)
284 (180, 360, 45)
285 (180, 360, 46)
286 (180, 360, 47)
287 (180, 360, 48)
288 (180, 360, 49)
289 (180, 360, 50)
290 (180, 360, 51)
291 (180, 360, 52)
292 (180, 360, 53)
293 (

In [17]:
# Flatten data for saving to CSV
for d in range(0, pco2_stack.shape[2]):
    if d == 0:
        pco2_flat = pco2_stack[:,:,d]
    else:
        pco2_slice = pco2_stack[:,:,d]
        pco2_flat = np.concatenate((pco2_flat,pco2_slice), axis=0)

# Print shape of flat data as check
print(pco2_flat.shape)

# Save data to CSV
np.savetxt('pco2_SOCAT22.csv', pco2_flat, delimiter=',')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27